In [10]:
import csv
from math import sqrt
import os
from copy import deepcopy

In [11]:
notebook_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("lin_regr.ipynb")), os.pardir))
data_folder = "/data/"

In [12]:
airline_delay_causes = []
columns = []
target_variable = "DEP_DELAY"
target_col = []

Считывание данных из файла в array

In [13]:
with open(notebook_path+data_folder+'2017.csv', newline='\n') as csv_file:
	csv_reader = csv.reader(csv_file, delimiter=',')
	line_count = 0
	for row in csv_reader:
		if line_count == 0:
			columns = row.copy()
			line_count += 1
		else:
			airline_delay_causes.append(row)
			line_count += 1

In [14]:
for col in columns:
    print(columns.index(col), " ", col)

0   YEAR
1   MONTH
2   DAY_OF_MONTH
3   DAY_OF_WEEK
4   OP_UNIQUE_CARRIER
5   TAIL_NUM
6   OP_CARRIER_FL_NUM
7   ORIGIN
8   DEST
9   CRS_DEP_TIME
10   DEP_TIME
11   DEP_DELAY
12   TAXI_OUT
13   TAXI_IN
14   CRS_ARR_TIME
15   ARR_TIME
16   ARR_DELAY
17   CANCELLED
18   CANCELLATION_CODE
19   DIVERTED
20   CRS_ELAPSED_TIME
21   ACTUAL_ELAPSED_TIME
22   AIR_TIME
23   FLIGHTS
24   DISTANCE
25   CARRIER_DELAY
26   WEATHER_DELAY
27   NAS_DELAY
28   SECURITY_DELAY
29   LATE_AIRCRAFT_DELAY
30   


Отбор необходимых признаков, отбрасывание столбцов, у которых эти фичи находятся без значения

In [15]:
target_index = 11
# features = [1, 2, 9, 10, 12, 17, 19, 25, 26, 27, 28, 29]
features = [1, 2, 9, 10, 17, 19]
# features = [9, 25, 29]
data_train_features = []
data_train_target = []
for row in airline_delay_causes:
	tmp = []
	for col in features:
		if len(row[col]) == 0:
			break
		if len(row[target_index]) == 0:
			break
		else:
			tmp.append(float(row[col]))
	if len(tmp) == len(features):
		data_train_features.append(tmp)
		data_train_target.append(float(row[target_index]))
len(data_train_features)

489170

Функции линейной алгебры

In [16]:
def norm(V):
    ans = 0
    for i in V:
        ans += i**2
    return sqrt(ans)

def sumvc(X, a):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] += a
    return ans

def sumvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] += Y[i]
    return ans

def subvc(X, a):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] -= a
    return ans

def subvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] -= Y[i]
    return ans

def multvc(X, c):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] *= c
    return ans

def multvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] *= Y[i]
    return ans

def dotvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = 0
    for i in range(len(X)):
        ans += X[i] * Y[i]
    return ans

def summc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for row in range(cols_A)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] + c
    return ans

def summm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise Exception("Dim of matrix should be same!")
    rows_A = len(A)
    cols_B = len(B[0])
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            ans[i][j] = A[i][j] + B[i][j]
    return ans

def submc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for row in range(cols_A)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] - c
    return ans

def submm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise Exception("Dim of matrix should be same!")
    rows_A = len(A)
    cols_B = len(B[0])
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            ans[i][j] = A[i][j] - B[i][j]
    return ans

def multmc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for col in range(cols_A)] for row in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] * c
    return ans

def multmv(A, V):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    rows_V = len(V)
    if cols_A != rows_V:
        raise Exception("Cannot multiply the matrix and vector. Incorrect dimensions.")
    ans = [0 for row in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i] += A[i][j] * V[j]
    return ans

def multmm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])
    if cols_A != rows_B:
        raise Exception("Cannot multiply the two matrices. Incorrect dimensions.")
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                ans[i][j] += A[i][k] * B[k][j]
    return ans

def decvc(V, c):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] /= c
    return ans

def decvv(V, W):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] /= W[i]
    return ans

def sqrtv(V):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] = sqrt(ans[i])
    return ans

def powv(V, s):
    if s == 0:
        return [1 for el in range(len(V))]
    tmp = powv(V, int(s/2))
    if s % 2 == 0:
        return multvv(tmp, tmp)
    else:
        return multvv(multvv(tmp, tmp), V)

Функции метрик

In [17]:
def MSE(W, X, Y, alpha=None):
    if len(X[0]) != len(W) or len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = 0
    for i in range(len(X)):
        ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
    return ans

In [18]:
def MSE_l1(W, X, Y, alpha):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ans = 0
	for i in range(len(X)):
		ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
	return ans + (alpha*norm(W))/len(W)

In [19]:
def MSE_l2(W, X, Y, alpha):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ans = 0
	for i in range(len(X)):
		ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
	return ans + (alpha*norm(W)**2)/len(W)

In [20]:
def R2(W, X, Y):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ym = sum(Y)/len(Y)
	SSreg = 0
	for i in range(len(X)):
		SSreg += (dotvv(W, X[i]) - ym)**2
	SStot = 0
	for i in range(len(Y)):
		SStot += (Y[i] - ym)**2
	return 1 - (SSreg / SStot)

Функции градиентного спуска

In [21]:
def grad(method, h, W, X, Y, alpha):
	col_X = len(X[0])
	ans = [0 for i in range(col_X)]
	Wn = W.copy()
	for i in range(col_X):
		Wn[i] += h
		ans[i] = (method(Wn, X, Y, alpha) - method(W, X, Y, alpha))/h
		Wn[i] -= h
	return ans

In [31]:
def LeastSquareMethod(method, A, y, alpha, beta=None, gamma=None):
    col_A = len(A[0])
    W = [1. for row in range(col_A)]
    step = 1e-8
    h = 1e-8
    L = 0
    Lnew = method(W, A, y, alpha)
    num_steps = 2000
    while abs(L - Lnew) > 1e-6 and num_steps > 0:
        L = Lnew
        W = subvv(W, multvc(grad(method, h, W, A, y, alpha), step))
        Lnew = method(W, A, y, alpha)
        num_steps -= 1
    return W

In [23]:
def SGD(method, A, y, alpha=None, beta=None, gamma=None):
	col_A = len(A[0])
	W = [1. for row in range(col_A)]
	step = 1e-8
	h = 1e-8
	L = 0
	Lnew = method(W, A, y, alpha)
	num_steps = 50
	while abs(L - Lnew) > 1e-5 and num_steps > 0:
		for i in range(len(A)):
			L = Lnew
			W = subvv(W, multvc(grad(method, h, W, [A[i]], [y[i]], alpha), step))
			Lnew = method(W, A, y, alpha)
		num_steps -= 1
	return W

In [24]:
def AdaGrad(method, A, y, alpha=None, beta=None, gamma=None):
	col_A = len(A[0])
	W = [1. for row in range(col_A)]
	step = 1e-1
	h = 1e-2
	eps = 1
	L = method(W, A, y, alpha)
	Gnii = [0 for i in range(col_A)]
	temp = grad(method, h, W, A, y, alpha)
	for i in range(col_A):
		Gnii[i] = (temp[i])**2
		W[i] -= step*temp[i]/sqrt(Gnii[i] + eps)
	Lnew = method(W, A, y, alpha)
	num_steps = 2500
	while abs(L - Lnew) > 1e-1 and num_steps > 0:
		L = Lnew
		temp = grad(method, h, W, A, y, alpha)
		for i in range(col_A):
			Gnii[i] += (temp[i])**2
			W[i] -= step*temp[i]/sqrt(Gnii[i] + eps)
		Lnew = method(W, A, y, alpha)
		num_steps -= 1
	return W

In [25]:
def RMSProp(method, A, y, alpha=None, beta=None, gamma=0.5):
    if gamma is None:
        gamma = 0.5
    col_A = len(A[0])
    W = [1. for row in range(col_A)]
    step = 1e-3
    h = 1e-3
    eps = 1e-3
    L = method(W, A, y, alpha)
    Eg = 0
    tmp = 0
    tmpnew = grad(method, h, W, A, y, alpha)
    Eg = multvc(powv(tmpnew, 2), 1-gamma)
    W = subvv(W, decvv(multvc(tmpnew, step), sqrtv(sumvc(Eg, eps))))
    Lnew = method(W, A, y, alpha)
    num_steps = 3500
    while abs(L - Lnew) > 1e-2 and num_steps > 0:
        L = Lnew
        tmp = tmpnew
        tmpnew = grad(method, h, W, A, y, alpha)
        Eg = sumvv(multvc(powv(tmp, 2), gamma), multvc(powv(tmpnew, 2), 1-gamma))
        W = subvv(W, decvv(multvc(tmpnew, step), sqrtv(sumvc(Eg, eps))))
        Lnew = method(W, A, y, alpha)
        num_steps -= 1
    return W

In [26]:
def Adam(method, A, y, alpha=None, gamma=0.9, beta=[0.95, 0.95]):
	if gamma is None:
		gamma = 0.9
	if beta is None:
		beta=[0.95, 0.95]
	col_A = len(A[0])
	W =[1. for row in range(col_A)]
	step = 1e-4
	h = 1e-3
	eps = 1e-3
	L = 0
	Lnew = method(W, A, y, alpha)
	tmp = grad(method, h, W, A, y, alpha)
	mt = multvc(tmp, 1-beta[0])
	_mt = decvc(mt, 1-beta[0])
	vt = multvc(powv(tmp, 2), 1-beta[1])
	_vt = decvc(vt, 1-beta[1])
	W = subvv(W, decvv(multvc(_mt, step), sqrtv(sumvc(_vt, eps))))
	L = Lnew
	Lnew = method(W, A, y, alpha)
	num_steps = 4000
	while abs(L - Lnew) > 1e-3 and num_steps > 0:
		L = Lnew
		tmp = grad(method, h, W, A, y, alpha)
		mt = sumvv(multvc(mt, beta[0]), multvc(tmp, 1-beta[0]))
		_mt = decvc(mt, 1-beta[0])
		vt = sumvv(multvc(vt, beta[1]), multvc(powv(tmp, 2), 1-beta[1]))
		_vt = decvc(vt, 1-beta[1])
		W = subvv(W, decvv(multvc(_mt, step), sqrtv(sumvc(_vt, eps))))
		Lnew = method(W, A, y, alpha)
		num_steps -= 1
	return W

Проверка работы написанных методов

In [27]:
alpha = 0.8

In [28]:
W = LeastSquareMethod(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (LSM) = ", W)
print("LSM MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("LSM R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (LSM) =  [0.9960995730343711, 0.9926346144393392, -0.0052154435543343425, 0.0016826163196128618, 0.999999868172381, 0.9999984121932357]
LSM MSE (train) =  2075.8892509715474
LSM R2  (train) =  0.9919665609015877
LSM MSE (test) =  3690.8990275526553
LSM R2  (test) =  0.9753011275538856


In [29]:
W = LeastSquareMethod(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (LSM) = ", W)
print("LSM MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("LSM R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (LSM) =  [0.9960991777934396, 0.9926342279768505, -0.005215435542140767, 0.0016826124242470542, 0.9999994697677721, 0.9999980173465701]
LSM MSE (train) =  2075.8892291786265
LSM R2  (train) =  0.9919665695895643
LSM MSE (test) =  3690.899013578265
LSM R2  (test) =  0.9753011183865042


In [30]:
W = SGD(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (SGD) = ", W)
print("SGD MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("SGD R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (SGD) =  [0.9767788640069827, 0.9569137232202143, -0.04171842779832985, 0.031776168422189666, 0.9999663673240918, 0.999945318684391]
SGD MSE (train) =  2083.542365121791
SGD R2  (train) =  0.9411037583164815
SGD MSE (test) =  4269.536261750089
SGD R2  (test) =  0.860223539374155


In [32]:
W = SGD(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (SGD) = ", W)
print("SGD MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("SGD R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (SGD) =  [0.9766811923975346, 0.9568177771588064, -0.041717396852635646, 0.03177632592747637, 0.9998666762150713, 0.9998456289685401]
SGD MSE (train) =  2083.528833862823
SGD R2  (train) =  0.9411070938539916
SGD MSE (test) =  4269.533763502152
SGD R2  (test) =  0.8602219440598152


In [33]:
W = AdaGrad(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (AdaGrad) = ", W)
print("AdaGrad MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("AdaGrad R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (AdaGrad) =  [0.012839853893695232, 0.012894829634810492, 0.002533774641954997, 0.004635690205356807, -0.004999999994089776, -0.014370021190601628]
AdaGrad MSE (train) =  2008.6060146942868
AdaGrad R2  (train) =  0.9922200698502885
AdaGrad MSE (test) =  3773.598139948191
AdaGrad R2  (test) =  0.9330724444048568


In [34]:
W = AdaGrad(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (AdaGrad) = ", W)
print("AdaGrad MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("AdaGrad R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (AdaGrad) =  [0.012846934300439136, 0.012898564238540884, 0.002533758474932849, 0.004635676121061766, 0.0454594756634639, -0.012611935473927319]
AdaGrad MSE (train) =  2008.6063164073314
AdaGrad R2  (train) =  0.9922200843895486
AdaGrad MSE (test) =  3773.5995550703256
AdaGrad R2  (test) =  0.9330735887075834


In [35]:
W = RMSProp(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [-0.1282746180461945, 0.026593967596313956, -0.040398098670400465, 0.04853623838478936, -0.0005000000028995327, -0.07381834830662207]
RMSProp MSE (train) =  1919.235336487971
RMSProp R2  (train) =  0.9410633964551341
RMSProp MSE (test) =  4147.037061167451
RMSProp R2  (test) =  0.8766233163137869


In [36]:
W = RMSProp(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [-0.1282746180461945, 0.026593967596313956, -0.040398098670400465, 0.04853623838478936, -0.0005000000028995327, -0.07381834830662207]
RMSProp MSE (train) =  1919.235336487971
RMSProp R2  (train) =  0.9410633964551341
RMSProp MSE (test) =  4147.037061167451
RMSProp R2  (test) =  0.8766233163137869


In [37]:
W = RMSProp(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha=0.5)
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [-0.13299528810048913, 0.028235232232833245, -0.040394053070609746, 0.04853847612882452, -0.0004999999976010491, -0.15100410821903637]
RMSProp MSE (train) =  1919.2124489846044
RMSProp R2  (train) =  0.9410504809288357
RMSProp MSE (test) =  4146.915028103956
RMSProp R2  (test) =  0.8766492615510101


In [38]:
W = Adam(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha=0.95, gamma=0.95, beta=[0.95, 0.95])
print("\nW (Adam) = ", W)
print("Adam MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("Adam R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (Adam) =  [-0.0013646780915485426, -0.0007490216235978604, -0.0033792158601944414, 0.009847250300560804, -0.0007818810409359904, -0.004373380923161368]
Adam MSE (train) =  1988.7776069910058
Adam R2  (train) =  0.9927093228484888
Adam MSE (test) =  3841.5518708807285
Adam R2  (test) =  0.9213626463397145


In [39]:
W = Adam(MSE_l2,data_train_features[:1000], data_train_target[:1000], alpha, gamma=0.95, beta=[0.9, 0.9])
print("\nW (Adam) = ", W)
print("Adam MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("Adam R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (Adam) =  [-0.04658733330257577, -0.019639268272707077, -0.0410710362486944, 0.048225087733778586, 0.003212863772234631, -0.06572414421194928]
Adam MSE (train) =  1919.2865334050412
Adam R2  (train) =  0.9441976334835918
Adam MSE (test) =  4194.796127046099
Adam R2  (test) =  0.864794574705339


Функция для кросс-валидации и работа с нею

In [40]:
def CrossValidation(methodGD, methodReg, X, Y, alpha=None, beta=None, gamma=None, folds=5):
    Wf = []
    onefold = int(1000/folds)
    for i in range(folds):
        Wf.append(methodGD(methodReg, X[:i*onefold] + X[(i+1)*onefold:], 
                                      Y[:i*onefold] + Y[(i+1)*onefold:], alpha, beta, gamma))
    W = Wf[0].copy()
    for i in range(1, len(Wf)):
        W = sumvv(W, Wf[i])
    W = decvc(W, len(Wf))
    MSE_train = []
    MSE_test = []
    R2_train = []
    R2_test = []
    for i in range(folds):
        MSE_train.append(MSE(W, X[:i*onefold] + X[(i+1)*onefold:], 
                                Y[:i*onefold] + Y[(i+1)*onefold:]))
        MSE_test.append(MSE(W, X[i*onefold:(i+1)*onefold], 
                                Y[i*onefold:(i+1)*onefold]))
        R2_train.append(R2(W, X[:i*onefold] + X[(i+1)*onefold:], 
                              Y[:i*onefold] + Y[(i+1)*onefold:]))
        R2_test.append(R2(W, X[i*onefold:(i+1)*onefold], 
                             Y[i*onefold:(i+1)*onefold]))
    print("\nW " + str(methodGD.__name__) + " (" + str(methodReg.__name__) + ") = ", W)
    print(str(methodGD.__name__) + " MSE (train) = ", sum(MSE_train)/len(MSE_train))
    print(str(methodGD.__name__) + " R2  (train) = ", sum(R2_train)/len(R2_train))
    print(str(methodGD.__name__) + " MSE (test) = ", sum(MSE_test)/len(MSE_test))
    print(str(methodGD.__name__) + " R2  (test) = ", sum(R2_test)/len(R2_test))

In [41]:
CrossValidation(LeastSquareMethod, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha, beta=None, gamma=None)


W LeastSquareMethod (MSE_l1) =  [0.9953852417013878, 0.9912792999607063, -0.030614691657137882, 0.027037387329482954, 0.999998666602778, 0.9999957553703098]
LeastSquareMethod MSE (train) =  2001.5915262803435
LeastSquareMethod R2  (train) =  0.9737650094231194
LeastSquareMethod MSE (test) =  2001.5915262803433
LeastSquareMethod R2  (test) =  0.9042721596329862


In [42]:
CrossValidation(LeastSquareMethod, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha, beta=None, gamma=None)


W LeastSquareMethod (MSE_l2) =  [0.995381261637931, 0.9912753452806555, -0.030614617070114036, 0.027037356307209846, 0.9999946689767512, 0.9999917580214059]
LeastSquareMethod MSE (train) =  2001.5911740598544
LeastSquareMethod R2  (train) =  0.9737651429346539
LeastSquareMethod MSE (test) =  2001.5911740598556
LeastSquareMethod R2  (test) =  0.9042724935119153


In [43]:
CrossValidation(SGD, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha, beta=None, gamma=None)


W SGD (MSE_l1) =  [0.9806685510328548, 0.9642665064292528, -0.045983597620300486, 0.036235337506915856, 0.999973134558266, 0.9999563550967092]
SGD MSE (train) =  2073.2146636422162
SGD R2  (train) =  0.9348553744410957
SGD MSE (test) =  2073.214663642216
SGD R2  (test) =  0.8926805023035312


In [44]:
CrossValidation(SGD, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha=0.5)


W SGD (MSE_l2) =  [0.9806294278036679, 0.9642279547393404, -0.045983157752615834, 0.03623536775029057, 0.9999333368553035, 0.999916557827573]
SGD MSE (train) =  2073.20945248313
SGD R2  (train) =  0.9348567142910585
SGD MSE (test) =  2073.2094524831314
SGD R2  (test) =  0.8926832589218426


In [45]:
CrossValidation(AdaGrad, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha=0.9)


W AdaGrad (MSE_l1) =  [0.01283841753336143, 0.012894925337333382, 0.0025341001248489563, 0.004634246959757451, -0.004999999999371232, -0.014153528567801523]
AdaGrad MSE (train) =  2008.6101061069999
AdaGrad R2  (train) =  0.9916135406324095
AdaGrad MSE (test) =  2008.6101061069985
AdaGrad R2  (test) =  0.9534155674793631


In [46]:
CrossValidation(AdaGrad, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha=0.9)


W AdaGrad (MSE_l2) =  [0.01284695009369142, 0.012899426459130694, 0.002534079876109576, 0.0046342291489263835, 0.035910124483830075, -0.012707697146466907]
AdaGrad MSE (train) =  2008.610421869816
AdaGrad R2  (train) =  0.9916135627399285
AdaGrad MSE (test) =  2008.6104218698154
AdaGrad R2  (test) =  0.9534152973564417


In [47]:
CrossValidation(RMSProp, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha=0.5)


W RMSProp (MSE_l1) =  [-0.13338628345341777, 0.02823915908127107, -0.043202102209834704, 0.05091191647403748, -0.0004999951861083364, -0.2996932807943412]
RMSProp MSE (train) =  1918.9718909750045
RMSProp R2  (train) =  0.9353786305784725
RMSProp MSE (test) =  1918.9718909750034
RMSProp R2  (test) =  0.8651547223215245


In [48]:
CrossValidation(RMSProp, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha=0.8)


W RMSProp (MSE_l2) =  [-0.13329383207825635, 0.02834872417466955, -0.043203303244974754, 0.05091179303308062, -0.0004999999681632409, -0.22174037927381568]
RMSProp MSE (train) =  1918.979207445915
RMSProp R2  (train) =  0.9353812053271291
RMSProp MSE (test) =  1918.9792074459133
RMSProp R2  (test) =  0.8651590345051391


In [49]:
CrossValidation(Adam, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha=0.5)


W Adam (MSE_l1) =  [-0.02157389768030381, -0.017688393911576377, -0.04187885699729514, 0.04889887943799223, -0.000661841787273847, -0.05931671710619311]
Adam MSE (train) =  1919.6278185227172
Adam R2  (train) =  0.9412147632619063
Adam MSE (test) =  1919.627818522718
Adam R2  (test) =  0.8794860037475056


In [50]:
CrossValidation(Adam, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha=0.9)


W Adam (MSE_l2) =  [-0.02163097536068504, -0.017722661305183927, -0.04187887558975095, 0.048899407390149495, -0.0005386130328077857, -0.1055236501210488]
Adam MSE (train) =  1919.6239162250993
Adam R2  (train) =  0.9412127260336955
Adam MSE (test) =  1919.6239162250974
Adam R2  (test) =  0.8794828844260352
